# SQL Tasks
## Part 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.

In [1]:
import sqlite3
import pandas as pd

database = "sqlite_db_pythonsqlite.db"

In [2]:

def connect_query(db_file, query):
    """
    create a database connection to the SQLite database specified by the db_file,
    execute specified query on the database connection,
    convert results to pandas Dataframe.
    
    Args: 
        db_file: database file
        query: SQL query string
        
    Return: 
        pandas dataframe
    """

    # connect to database
    conn = sqlite3.connect(db_file)

    # create cursor and execute the query
    # extract resulting table as rows and column names as col
    with conn:
        cur = conn.cursor()
        result = cur.execute(query)
        rows = result.fetchall()
        col = [description[0] for description in cur.description]

    # create and return pandas Dataframe using rows and column names
    return pd.DataFrame(rows, columns = col)


In [3]:
query_test1 = 'SELECT * FROM Facilities'
connect_query(database, query_test1)


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


### Q10: Produce a list of facilities with a total revenue less than 1000. 

The output of facility name and total revenue, sorted by revenue. 
Remember that there's a different cost for guests and members!

In [4]:
query_10 = '''
WITH revenue_per_booking AS (
    SELECT f.name AS facility_name,
        m.surname || ', ' || m.firstname AS member_name,
        CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
        ELSE f.membercost * b.slots END AS revenue
    FROM Bookings AS b
    LEFT JOIN Facilities AS f ON b.facid = f.facid
    LEFT JOIN Members AS m ON b.memid = m.memid)
    
SELECT *
FROM (
    SELECT facility_name, 
        SUM(revenue) AS total_revenue
    FROM revenue_per_booking
    GROUP BY facility_name) AS sub
WHERE total_revenue < 1000
ORDER BY total_revenue DESC;
'''

connect_query(database, query_10)


,facility_name,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [5]:
query_11 = '''
SELECT 
    m1.surname || ', ' || m1.firstname AS member_name, 
    CASE WHEN m1.recommendedby >0 THEN m2.surname || ', ' || m2.firstname 
    ELSE NULL END AS recommendedby_name
FROM Members AS m1
LEFT JOIN Members AS m2 ON m1.recommendedby = m2.memid
WHERE m1.memid > 0
ORDER BY member_name;
'''

connect_query(database, query_11)

,member_name,recommendedby_name
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Farrell, David",None
9,"Farrell, Jemima",None


### Q12: Find the facilities with their usage by member, but not guests

In [6]:
query_12 = '''
SELECT DISTINCT f.name AS facility_name, 
    m.surname || ', ' || m.firstname AS member_name, 
    COUNT(*) as usage
FROM Bookings AS b
LEFT JOIN Members AS m ON b.memid = m.memid
LEFT JOIN Facilities AS f on b.facid = f.facid
WHERE b.memid > 0
GROUP BY facility_name, member_name
ORDER BY facility_name, member_name;
'''

connect_query(database, query_12)

,facility_name,member_name,usage
0,Badminton Court,"Bader, Florence",9
1,Badminton Court,"Baker, Anne",10
2,Badminton Court,"Baker, Timothy",7
3,Badminton Court,"Boothe, Tim",12
4,Badminton Court,"Butters, Gerald",20
...,...,...,...
197,Tennis Court 2,"Smith, Darren",19
198,Tennis Court 2,"Smith, Jack",1
199,Tennis Court 2,"Smith, Tracy",2
200,Tennis Court 2,"Stibbons, Ponder",31


### Q13: Find the facilities usage by month, but not guests

In [7]:
query_13 = '''
SELECT DISTINCT f.name AS facility_name, 
    strftime('%m', starttime) AS month,
    COUNT(*) as usage
FROM Bookings AS b
LEFT JOIN Members AS m ON b.memid = m.memid
LEFT JOIN Facilities AS f on b.facid = f.facid
WHERE b.memid > 0
GROUP BY facility_name, month
ORDER BY facility_name, month;
'''

connect_query(database, query_13)

,facility_name,month,usage
0,Badminton Court,07,51
1,Badminton Court,08,132
2,Badminton Court,09,161
3,Massage Room 1,07,77
4,Massage Room 1,08,153
5,Massage Room 1,09,191
6,Massage Room 2,07,4
7,Massage Room 2,08,9
8,Massage Room 2,09,14
9,Pool Table,07,103
